In [6]:
#coding=utf-8
'''
预处理未标注的word数据
'''
#coding=utf-8
import pandas as pd
import  numpy as np
import csv
from sklearn import preprocessing
from  sklearn.preprocessing import MinMaxScaler
df  = pd.read_table('traindata.txt',sep='\t',quoting=csv.QUOTE_NONE,encoding='utf-8')
del df['文章名称'],df['文本内容']
del df['字体']
# df = df[df['段落角色']!='-']
df.loc[df['中文比例']=='非数字','中文比例'] = 1
df[df.isnull().values==True]

# df.to_csv('train_2.csv',index=False,header=True,encoding='utf-8')

1             85042
0             25207
0.5            1736
0.6666667      1575
0.75           1237
0.8            1037
0.995          1016
0.5714286       920
0.4             898
0.625           868
-               844
0.3333333       820
0.8333333       698
0.8571429       669
0.7             668
0.6             635
0.875           601
0.99            544
0.9             537
0.25            533
0.7272727       520
0.8888889       517
非数字             490
0.98            465
0.7857143       450
0.7142857       437
0.9090909       431
0.7692308       410
0.985           408
0.5555556       392
              ...  
0.6768293         1
0.5521472         1
0.3675214         1
0.3895349         1
0.5037594         1
0.1703297         1
0.2210526         1
0.4673913         1
0.3282051         1
0.6793478         1
0.07070707        1
0.2393162         1
0.3697479         1
0.5346535         1
0.2047244         1
0.06617647        1
0.3596491         1
0.613924          1
0.04972376        1


In [2]:
df.loc[df['字号'].isnull()]

,段落角色,等号,关键字,字体,字号,字形,字数,编号,编号位置,word对象,行距,缩进,段前距,段后距,对齐方式,大纲级别,标点,中文比例,邮箱符号


In [13]:
df = pd.read_csv('train.csv',encoding='utf-8')
df.columns

Index(['关键字_abstract', '关键字_keywords', '关键字_key words', '关键字_null',
       '关键字_关　键　词', '关键字_关键字', '关键字_关键词', '关键字_图', '关键字_摘　　要', '关键字_摘　要',
       ...
       '标点_True', '字形', '字数', '编号', '中文比例', '邮箱符号', '文章名称', '字号', 'd2v_id',
       '段落角色'],
      dtype='object', length=771)

In [9]:

def clean_big_data(x):
    try:
          if (float(x)>10):
              return 'null'
          else :
               return '数字1'
    except:
          return x
     

In [10]:
import regex as re
def clean_value_change(x):
    if (x!='null') & ('.' in str(x)):  
#         x = '长度'+str(len(str(x).split('.')))
        x = '长度'+str(len(str(x).split('.')))
    elif (x!='null') &((')' in str(x))|('）' in str(x))|('（' in str(x))|(']' in str(x))):  
        x =  '括号'
    r1 = re.compile(u'[0-9][、|.|,|．|，]')
    r2 = re.compile(u'[④|①|②|③|⑤|⑦|△|⑥]')
    r3 = re.compile(u'[一|二|三|四|五|六|第]')
#     r4 = re.compile(u'[|二|三|四|五|六|第]')
    try:
        if(len(r1.findall(x))!=0):
            return '数字+符号'
        elif(len(r2.findall(x))!=0):
            return '罗马符号'
        elif(len(r3.findall(x))!=0):
            return '一级'
        else:
            return x
    except:
            return x

In [11]:
import regex as re
def clean_x_null(x):
    if(x=='-'):
        return x
    r1 = re.compile(u'[数字|括号|长度|一级|罗马符号|六|第]')
    try:
        if(len(r1.findall(x))!=0):
                return x
        else:
                return 'null'
    except:
        pass

In [12]:
df['编号处理'] = df['编号'].map(lambda x: clean_big_data(x))
df['编号处理'] = df['编号处理'].map(lambda x: clean_value_change(x))
df['编号处理'] = df['编号处理'].map(lambda x: clean_x_null(x))
df_bianhao = pd.get_dummies(df['编号处理'],prefix='编号')
print(df_bianhao.columns)
len_df_bianhao = len(df_bianhao.loc[(df_bianhao['编号_-']==1)])
df_bianhao.loc[(df_bianhao['编号_-']==1)]=[['-','-','-','-','-','-','-','-','-','-']]*len_df_bianhao

del df_bianhao['编号_-']
del df['编号处理']
df = pd.concat([df,df_bianhao],axis=1)
print(df.columns)
df['关键字'] = df['关键字'].str.replace('　','')
df['关键字'] = df['关键字'].str.replace(r'key.*words','keywords')
df.to_csv('../data/train_bianhao_clean.csv',index=False)

Index(['编号_-', '编号_null', '编号_一级', '编号_括号', '编号_数字+符号', '编号_数字1', '编号_罗马符号',
       '编号_长度2', '编号_长度3', '编号_长度4'],
      dtype='object')
Index(['段落角色', '等号', '关键字', '字体', '字号', '字形', '字数', '编号', '编号位置', 'word对象',
       '行距', '缩进', '段前距', '段后距', '对齐方式', '大纲级别', '标点', '中文比例', '邮箱符号',
       '编号_null', '编号_一级', '编号_括号', '编号_数字+符号', '编号_数字1', '编号_罗马符号', '编号_长度2',
       '编号_长度3', '编号_长度4'],
      dtype='object')


In [7]:
print(len(df))
print(df.loc[df['字号'].isnull(),'文本内容'])

82762
Series([], Name: 文本内容, dtype: object)


In [8]:
df['关键字'].value_counts()

null        70018
图            5029
表            1832
摘要           1459
-            1359
关键词          1148
abstract      964
keywords      893
关键字            60
Name: 关键字, dtype: int64

In [56]:
"我 站在".replace(" ","")

'我站在'

In [ ]:
df['编号'.isnull]

In [19]:
s = pd.Series(list('abc'))
s

0    a
1    b
2    c
dtype: object

In [20]:
s1 = pd.get_dummies(s,columns=('a','c','b'))
s1

,a,b,c
0,1,0,0
1,0,1,0
2,0,0,1
